In [11]:
!pip install playwright



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


TimeoutError: Timeout 30000.0ms exceeded.

In [12]:
from playwright.async_api import async_playwright
import asyncio

class LegoAPI:
    root_url = 'https://www.brickeconomy.com'

    def __init__(self, set_list):
        self.set_list = set_list

    async def start(self):
        try:
            with open(self.set_list, 'r') as f:
                set_list = [line.rstrip() for line in f.readlines()]
        except Exception as e:
            print("Error opening input file")
            raise e

        async with async_playwright() as p:
            browser = await p.chromium.launch()
            page = await browser.new_page()

            for set_num in set_list:
                search_url = f"{self.root_url}/search?query={set_num}"
                await page.goto(search_url)
                await page.wait_for_load_state("networkidle")

                try:
                    possible_links = await page.query_selector_all(
                        'table#ContentPlaceHolder1_ctlSetsOverview_GridViewSets tr td.hidden-xs.ctlsets-image a')
                except Exception as e:
                    raise ValueError(f"Error parsing HTML: {e}")

                if not possible_links:
                    raise ValueError(f"No links found for set number: {set_num}")

                for link in possible_links:
                    href = await link.get_attribute("href")
                    test_num = href.split("/")[2].split("-")[0]
                    if test_num == set_num:
                        set_details = href.split("/")[2:4]
                        await page.goto(self.root_url + href)
                        await page.wait_for_load_state("networkidle")
                        await self.parse_set(page, set_details)

            await browser.close()

    async def parse_set(self, page, set_details):
        main_div = await page.query_selector('div#ContentPlaceHolder1_PanelSetPricing')

        if not main_div:
            raise ValueError("Main div not found in HTML")

        price_list = (await main_div.query_selector_all('div.row.rowlist div')).text_content().split('\n')

        if not price_list:
            raise ValueError("Price list not found in HTML")

        item_details = {" ".join(set_details): price_list}
        print(item_details)

In [13]:
lego_api = LegoAPI('set_list.txt')
await lego_api.start()

ValueError: No links found for set number: 001-1